# Text Similarity

My team and I decided to solve Fake Detection task through finding text similarities of the current news and news from white-list sources

## Imports

In [1]:
!pip install transformers

In [1]:
import pandas as pd
import numpy as np
import torch

from nltk.corpus import stopwords

from sklearn.metrics.pairwise import cosine_similarity, manhattan_distances, euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer

from transformers import AutoTokenizer, AutoModel

## Loading data

In [2]:
data = pd.read_json("data/ixbt.json")

In [3]:
data.head()

,title,subtitle,url,time,text,tags,sources
0,Первый на рынке смартфон с новой 108-мегапиксе...,Он получит экран AMOLED,https://www.ixbt.com/news/2022/04/01/108-realm...,01.04.2022 23:18,"Информации о смартфоне Realme 9 4G, который пе...",[Realme],[Gizmochina]
1,Стоечные сетевые хранилища Qnap TS-873AeU пост...,Они рассчитаны на восемь накопителей,https://www.ixbt.com/news/2022/04/01/qnap-ts-8...,01.04.2022 21:21,Компания Qnap Systems представила серию хранил...,[Qnap ],[Qnap]
2,Китайский космический корабль «Тяньчжоу-2» сош...,Он использовался для доставки грузов на китайс...,https://www.ixbt.com/news/2022/04/01/kitajskij...,01.04.2022 21:15,Как пишет Интерфакс со ссылкой на Центральное ...,[],[Интерфакс]
3,Зачем делать новый смартфон лучше старого? Sam...,Платформа Exynos 850 сохранится,https://www.ixbt.com/news/2022/04/01/samsung-g...,01.04.2022 21:15,Компания Samsung готовит ещё один очень доступ...,[Samsung],[GSM Arena]
4,Ford и GM остановят производство на двух завод...,Мировая автомобильная промышленность испытывае...,https://www.ixbt.com/news/2022/04/01/ford-i-gm...,01.04.2022 20:51,Компании Ford Motor и General Motors (GM) на с...,"[Ford, GM]",[Reuters]


In [4]:
data.shape

(4044, 7)

## RuBERT

In [5]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-conversational")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-conversational")

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# Preprocessing
tokens = {'input_ids': [], 'attention_mask': []}

for sentence in data.text[:20]:
    # encode each sentence and append to dictionary
    new_tokens = tokenizer.encode_plus(sentence, max_length=512,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

# reformat list of tensors into single tensor
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

In [7]:
idfs_score = dict()

for line in tokens['input_ids']:
    u_line_tokens = torch.unique(line)
    for token in u_line_tokens:
        try:
            idfs_score[token.item()] += 1
        except KeyError:
            idfs_score[token.item()] = 1

In [8]:
idfs_score = {item[0]: np.log(tokens['input_ids'].shape[0] / item[1]) for item in idfs_score.items()}

In [10]:
%%time
# Using model
outputs = model(**tokens)

Wall time: 56.6 s


In [11]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [12]:
embeddings = outputs.last_hidden_state
embeddings

tensor([[[-0.0123,  0.5137,  0.7827,  ...,  1.2225,  0.5043, -0.3281],
         [-0.3710, -0.4433,  1.1205,  ...,  1.0690,  1.4072, -0.4002],
         [-0.9653, -0.4438,  0.9468,  ...,  0.3695,  0.4446, -0.8369],
         ...,
         [-0.6415,  0.4246,  1.6744,  ...,  1.6243,  0.9710, -0.7326],
         [-0.6097,  0.2388,  1.8754,  ...,  1.5156,  0.5597, -0.7159],
         [-0.4524,  0.0370,  1.3068,  ...,  1.3169,  0.5921,  0.1661]],

        [[-0.8850, -0.1318, -0.2900,  ...,  0.8480,  0.7990, -0.4450],
         [-0.6553, -0.4653,  0.5654,  ...,  1.3518,  0.4975, -0.9408],
         [-0.5849,  0.3679,  0.8588,  ..., -0.8018,  0.4140, -1.5484],
         ...,
         [-0.7501, -0.5068, -0.2334,  ..., -0.0671,  1.4792, -0.2848],
         [-1.0348, -0.5302,  0.3802,  ..., -0.2080,  1.5405, -0.5321],
         [-0.3353, -0.0545, -0.1519,  ...,  1.0112,  0.9447, -0.1615]],

        [[-0.2809,  0.4852,  0.4439,  ...,  0.6116,  0.4189, -0.2743],
         [-0.6677,  0.3747,  0.2640,  ...,  0

In [13]:
embeddings.shape

torch.Size([20, 512, 768])

In [14]:
attention_mask = tokens['attention_mask']
# Reshaping attenion mask
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
# Applying attention mask
masked_embeddings = embeddings * mask

# Getting average vector
summed = torch.sum(masked_embeddings, 1)

summed_mask = torch.clamp(mask.sum(1), min=1e-9)

text_vectors = summed / summed_mask

In [15]:
text_vectors

tensor([[-0.4340, -0.2074,  0.6403,  ...,  1.0288,  0.5380, -0.2410],
        [-0.5237, -0.2945,  0.0657,  ...,  0.8257,  0.7562, -0.2475],
        [-0.2741, -0.1203,  0.4638,  ...,  0.7611,  0.2944,  0.0548],
        ...,
        [-0.4745,  0.1994,  0.7839,  ...,  0.5246,  0.3847, -0.3414],
        [-0.3081, -0.2650,  0.6648,  ...,  0.6837,  0.1493, -0.2319],
        [-0.5242, -0.1639,  0.6322,  ...,  0.8743,  0.3982, -0.2809]],
       grad_fn=<DivBackward0>)

In [16]:
text_vectors.shape

torch.Size([20, 768])

In [17]:
cosine_similarities_dct = dict()

for i in range(embeddings.shape[0]):
    for j in range(i + 1, embeddings.shape[0]):
        first_vector = text_vectors[i].detach().numpy().reshape(1, -1)
        second_vector = text_vectors[j].detach().numpy().reshape(1, -1)
        cosine_similarities_dct[f'{i} - {j}'] = cosine_similarity(first_vector, second_vector)[0][0]

In [18]:
for pair, similarity in sorted(cosine_similarities_dct.items(), key=lambda x: x[1], reverse=True):
    print(f"Pair {pair} -> {similarity}")

Pair 3 - 17 -> 0.9160645008087158
Pair 3 - 19 -> 0.9086664915084839
Pair 3 - 18 -> 0.8823859691619873
Pair 17 - 19 -> 0.8762884140014648
Pair 8 - 13 -> 0.8704392910003662
Pair 3 - 7 -> 0.8688899278640747
Pair 18 - 19 -> 0.8605533838272095
Pair 11 - 15 -> 0.8589591383934021
Pair 11 - 13 -> 0.8478221893310547
Pair 1 - 13 -> 0.8468063473701477
Pair 0 - 3 -> 0.8467146158218384
Pair 7 - 17 -> 0.843827486038208
Pair 13 - 15 -> 0.8434358239173889
Pair 0 - 17 -> 0.8427705764770508
Pair 4 - 15 -> 0.8415980935096741
Pair 0 - 15 -> 0.8410001993179321
Pair 10 - 18 -> 0.8375211954116821
Pair 4 - 11 -> 0.8359225988388062
Pair 7 - 18 -> 0.834682822227478
Pair 16 - 18 -> 0.8337271213531494
Pair 17 - 18 -> 0.8276875019073486
Pair 1 - 8 -> 0.8276330232620239
Pair 6 - 18 -> 0.8265410661697388
Pair 6 - 16 -> 0.825156033039093
Pair 0 - 19 -> 0.8242336511611938
Pair 3 - 16 -> 0.8226326704025269
Pair 8 - 19 -> 0.8197687864303589
Pair 16 - 19 -> 0.8197057843208313
Pair 0 - 11 -> 0.8193777203559875
Pair 7 - 19

In [19]:
print("Text #3:")
print(data.text[3])
print()
print("Text #17:")
print(data.text[17])

Text #3:
Компания Samsung готовит ещё один очень доступный смартфон. Galaxy F13 уже засветился в Geekbench, что позволяет нам заодно узнать его основные параметры. В частности, сердцем новинки послужит SoC Exynos 850 — самая простая и бюджетная однокристальная система Samsung среди актуальных. Напомним, в её конфигурацию входят восемь ядер Cortex-A55 с частотой 2 ГГц и GPU Mali-G52 MP1. В бенчмарке платформа набирает 157 и 587 баллов в однопоточном и многопоточном режимах соответственно. При этом у тестового аппарата было 4 ГБ ОЗУ. Для сравнения, Galaxy F12 основан на той же платформе и имеет те же 4 ГБ ОЗУ. То есть с точки зрения производительности разницы не будет вообще никакой. Что изменится, неясно. Если слухи верны, и Galaxy F13 будет копией Galaxy A13, то экран увеличится до 6,6 дюйма, а ёмкость аккумулятора снизится до 5000 мА•ч.

Text #17:
Компания Lenovo готовится пополнить ассортимент смартфонов Motorola моделью Moto G62. Когда случится официальная премьера — данных нет, но 

In [20]:
syntatic_text = "Samsung готовит ещё один очень бюджетный смартфон. Galaxy F13 уже засветился в Geekbench, что" +\
"позволяет нам заодно узнать его основные параметры. "       +\
"При тестах платформа набирает 157 и 587 баллов в однопоточном и многопоточном режимах соответственно. При этом у " +\
"этого тестового аппарата было 4 ГБ оперативной памяти. Например Galaxy F12 "        +\
"основан на той же платформе и имеет тот же объём памяти. То есть с точки зрения производительности разницы не будет "        +\
"вообще никакой. Если теория подтвердится и Galaxy F13 будет копией Galaxy A13, то экран увеличится, а ёмкость аккумулятора снизится."

In [21]:
syntatic_text

'Samsung готовит ещё один очень бюджетный смартфон. Galaxy F13 уже засветился в Geekbench, чтопозволяет нам заодно узнать его основные параметры. При тестах платформа набирает 157 и 587 баллов в однопоточном и многопоточном режимах соответственно. При этом у этого тестового аппарата было 4 ГБ оперативной памяти. Например Galaxy F12 основан на той же платформе и имеет тот же объём памяти. То есть с точки зрения производительности разницы не будет вообще никакой. Если теория подтвердится и Galaxy F13 будет копией Galaxy A13, то экран увеличится, а ёмкость аккумулятора снизится.'

In [22]:
syntatic_text_tokens = {'input_ids': 0, 'attention_mask': 0}

new_tokens = tokenizer.encode_plus(syntatic_text, max_length=512,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')

syntatic_text_tokens['input_ids'] = new_tokens['input_ids'][0].reshape(1, -1)
syntatic_text_tokens['attention_mask'] = new_tokens['attention_mask'][0].reshape(1, -1)

In [23]:
%%time
syntatic_text_output = model(**syntatic_text_tokens)

Wall time: 4.7 s


In [25]:
syntatic_text_embd = syntatic_text_output.last_hidden_state

In [26]:
syn_attention_mask = syntatic_text_tokens['attention_mask']
# Reshaping attenion mask
syn_mask = syn_attention_mask.unsqueeze(-1).expand(syntatic_text_embd.size()).float()
# Applying attention mask
syn_masked_embeddings = syntatic_text_embd * syn_mask
# Getting average vector
syn_summed = torch.sum(syn_masked_embeddings, 1)

syn_summed_mask = torch.clamp(syn_mask.sum(1), min=1e-9)

syn_text_vector = syn_summed / syn_summed_mask

In [27]:
syn_cosine_similarities_dct = dict()
text_vectors_with_synt = torch.vstack([text_vectors, syn_text_vector])

for i in range(text_vectors_with_synt.shape[0]):
    for j in range(i + 1, text_vectors_with_synt.shape[0]):
        first_vector = text_vectors_with_synt[i].detach().numpy().reshape(1, -1)
        second_vector = text_vectors_with_synt[j].detach().numpy().reshape(1, -1)
        syn_cosine_similarities_dct[f'{i} - {j}'] = {
            "Cosine measure": cosine_similarity(first_vector, second_vector)[0][0],
            "Manhattan distance": manhattan_distances(first_vector, second_vector)[0][0],
            "Euclidian distance": euclidean_distances(first_vector, second_vector)[0][0],
        }

In [28]:
for pair, similarity in sorted(syn_cosine_similarities_dct.items(), key=lambda x: x[1]['Cosine measure'], reverse=True):
    print(f"Pair {pair} -> {similarity}")

Pair 3 - 17 -> {'Cosine measure': 0.9160645, 'Manhattan distance': 80.24904120840074, 'Euclidian distance': 3.6523664}
Pair 3 - 19 -> {'Cosine measure': 0.9086665, 'Manhattan distance': 84.65269611473195, 'Euclidian distance': 3.8062818}
Pair 3 - 20 -> {'Cosine measure': 0.90118915, 'Manhattan distance': 93.22176944675448, 'Euclidian distance': 4.1853266}
Pair 3 - 18 -> {'Cosine measure': 0.88238597, 'Manhattan distance': 97.29390385841543, 'Euclidian distance': 4.4848604}
Pair 17 - 19 -> {'Cosine measure': 0.8762884, 'Manhattan distance': 99.04203205138037, 'Euclidian distance': 4.504569}
Pair 8 - 13 -> {'Cosine measure': 0.8704393, 'Manhattan distance': 107.3117418197362, 'Euclidian distance': 4.8723288}
Pair 3 - 7 -> {'Cosine measure': 0.8688899, 'Manhattan distance': 100.10682574861858, 'Euclidian distance': 4.5001836}
Pair 18 - 19 -> {'Cosine measure': 0.8605534, 'Manhattan distance': 108.21965754123812, 'Euclidian distance': 4.9223475}
Pair 11 - 15 -> {'Cosine measure': 0.8589591

In [30]:
print("Text #3:")
print(data.text[3])
print()
print("Text #17:")
print(data.text[17])
print()
print("Syntetic text: ")
print(syntatic_text)

Text #3:
Компания Samsung готовит ещё один очень доступный смартфон. Galaxy F13 уже засветился в Geekbench, что позволяет нам заодно узнать его основные параметры. В частности, сердцем новинки послужит SoC Exynos 850 — самая простая и бюджетная однокристальная система Samsung среди актуальных. Напомним, в её конфигурацию входят восемь ядер Cortex-A55 с частотой 2 ГГц и GPU Mali-G52 MP1. В бенчмарке платформа набирает 157 и 587 баллов в однопоточном и многопоточном режимах соответственно. При этом у тестового аппарата было 4 ГБ ОЗУ. Для сравнения, Galaxy F12 основан на той же платформе и имеет те же 4 ГБ ОЗУ. То есть с точки зрения производительности разницы не будет вообще никакой. Что изменится, неясно. Если слухи верны, и Galaxy F13 будет копией Galaxy A13, то экран увеличится до 6,6 дюйма, а ёмкость аккумулятора снизится до 5000 мА•ч.

Text #17:
Компания Lenovo готовится пополнить ассортимент смартфонов Motorola моделью Moto G62. Когда случится официальная премьера — данных нет, но 

## BERT pipeline

    Creating one pipeline for our all code above

In [31]:
class RuBERT:
    def __init__(self):
        self.model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-conversational")
        self.tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-conversational")
    
    def get_embeddings(self, texts: list, update_idf=False):
        tokens = {'input_ids': [], 'attention_mask': []}

        for sentence in texts:
            # encode each sentence and append to dictionary
            new_tokens = self.tokenizer.encode_plus(sentence, max_length=512,
                                                    truncation=True, padding='max_length',
                                                    return_tensors='pt')
            tokens['input_ids'].append(new_tokens['input_ids'][0])
            tokens['attention_mask'].append(new_tokens['attention_mask'][0])

        # reformat list of tensors into single tensor
        tokens['input_ids'] = torch.stack(tokens['input_ids'])
        tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
        # Running model
        outputs = self.model(**tokens)

        embeddings = outputs.last_hidden_state
        # Updating IDF if necessery
        if update_idf:
            # Calculating IDF score
            self.idfs_score = dict()

            for line in tokens['input_ids']:
                u_line_tokens = torch.unique(line)
                for token in u_line_tokens:
                    try:
                        self.idfs_score[token.item()] += 1
                    except KeyError:
                        self.idfs_score[token.item()] = 1

            self.idfs_score = {item[0]: np.log(tokens['input_ids'].shape[0] / item[1]) for item in self.idfs_score.items()}
        
        attention_mask = tokens['attention_mask']
        # Reshaping attenion mask
        mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
        # Applying attention mask
        masked_embeddings = embeddings * mask
        
        # Applying IDF to model word vectors
        for sent_ind in range(masked_embeddings.shape[0]):
            for word_ind in range(masked_embeddings.shape[1]):
                masked_embeddings[sent_ind][word_ind] *= self.idfs_score.get(tokens['input_ids'][sent_ind][word_ind].item(), 1)
        
        # Getting average vector
        summed = torch.sum(masked_embeddings, 1)

        summed_mask = torch.clamp(mask.sum(1), min=1e-9)

        text_vectors = summed / summed_mask

        return text_vectors

In [32]:
rubert_obj = RuBERT()

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
# Text preprocessing
# Deleting stop words
rus_stop_words = stopwords.words('russian')

def del_stop_words(text):
    words = text.split(' ')
    return ' '.join([i for i in words if i not in rus_stop_words])

In [34]:
data.text[:20].apply(del_stop_words)

0     Информации смартфоне Realme 9 4G, который перв...
1     Компания Qnap Systems представила серию хранил...
2     Как пишет Интерфакс ссылкой Центральное телеви...
3     Компания Samsung готовит ещё очень доступный с...
4     Компании Ford Motor General Motors (GM) следую...
5     С 4 апреля сети кинотеатров «Каро» начнут пока...
6     Как пишет ТАСС, Минобрнауки разослало вузам пи...
7     Разработчики бенчмарка AnTuTu опубликовали све...
8     Ассортимент компании Ugreen, специализирующейс...
9     Компании Epic Games, похоже, придётся отвечать...
10    Специалисты аналитической компании IDC подвели...
11    Компания EK, специализирующаяся выпуске компон...
12    Министерство цифрового развития, связи массовы...
13    Компания EK, специализирующаяся выпуске компон...
14    Мошенники начали активную рассылку электронных...
15    Робот Boston Dynamics Stretch, который способе...
16    Компания Google объявила запуске новой функции...
17    Компания Lenovo готовится пополнить ассорт

    Now we can apply model and check it's performance

In [35]:
%%time
text_embeddings = rubert_obj.get_embeddings(data.text[:20].apply(del_stop_words), update_idf=True)

Wall time: 53.7 s


In [36]:
cosine_similarities_dct = dict()

for i in range(text_embeddings.shape[0]):
    for j in range(i + 1, text_embeddings.shape[0]):
        first_vector = text_embeddings[i].detach().numpy().reshape(1, -1)
        second_vector = text_embeddings[j].detach().numpy().reshape(1, -1)
        cosine_similarities_dct[f'{i} - {j}'] = cosine_similarity(first_vector, second_vector)[0][0]

In [37]:
for pair, similarity in sorted(cosine_similarities_dct.items(), key=lambda x: x[1], reverse=True):
    print(f"Pair {pair} -> {similarity}")

Pair 3 - 17 -> 0.8810005187988281
Pair 0 - 17 -> 0.8714632987976074
Pair 8 - 13 -> 0.867717981338501
Pair 0 - 3 -> 0.8672029972076416
Pair 0 - 19 -> 0.8639278411865234
Pair 3 - 19 -> 0.8625789880752563
Pair 17 - 19 -> 0.8600724339485168
Pair 8 - 19 -> 0.8469281792640686
Pair 13 - 19 -> 0.8322530388832092
Pair 1 - 19 -> 0.8299097418785095
Pair 0 - 18 -> 0.8292346000671387
Pair 1 - 13 -> 0.8220564126968384
Pair 18 - 19 -> 0.8182384967803955
Pair 1 - 8 -> 0.8169050812721252
Pair 6 - 12 -> 0.8147690296173096
Pair 3 - 18 -> 0.8141442537307739
Pair 7 - 18 -> 0.8102940320968628
Pair 0 - 8 -> 0.8045006990432739
Pair 0 - 7 -> 0.8015919923782349
Pair 17 - 18 -> 0.8015601634979248
Pair 7 - 17 -> 0.8014261722564697
Pair 3 - 7 -> 0.7962417602539062
Pair 0 - 1 -> 0.7924342155456543
Pair 10 - 18 -> 0.7919483184814453
Pair 4 - 18 -> 0.791883111000061
Pair 8 - 17 -> 0.7914964556694031
Pair 1 - 3 -> 0.791325569152832
Pair 0 - 16 -> 0.7889103889465332
Pair 13 - 18 -> 0.788846492767334
Pair 15 - 18 -> 0.7

In [38]:
syntatic_text = "Samsung готовит ещё один очень бюджетный смартфон. Galaxy F13 уже засветился в Geekbench, что" +\
"позволяет нам заодно узнать его основные параметры. "       +\
"При тестах платформа набирает 157 и 587 баллов в однопоточном и многопоточном режимах соответственно. При этом у " +\
"этого тестового аппарата было 4 ГБ оперативной памяти. Например Galaxy F12 "        +\
"основан на той же платформе и имеет тот же объём памяти. То есть с точки зрения производительности разницы не будет "        +\
"вообще никакой. Если теория подтвердится и Galaxy F13 будет копией Galaxy A13, то экран увеличится, а ёмкость аккумулятора снизится."

In [39]:
syntatic_text

'Samsung готовит ещё один очень бюджетный смартфон. Galaxy F13 уже засветился в Geekbench, чтопозволяет нам заодно узнать его основные параметры. При тестах платформа набирает 157 и 587 баллов в однопоточном и многопоточном режимах соответственно. При этом у этого тестового аппарата было 4 ГБ оперативной памяти. Например Galaxy F12 основан на той же платформе и имеет тот же объём памяти. То есть с точки зрения производительности разницы не будет вообще никакой. Если теория подтвердится и Galaxy F13 будет копией Galaxy A13, то экран увеличится, а ёмкость аккумулятора снизится.'

In [40]:
%%time
syn_embed = rubert_obj.get_embeddings([del_stop_words(syntatic_text)], update_idf=False)

Wall time: 3.39 s


In [41]:
syn_cosine_similarities_dct = dict()
text_embed_with_syn = torch.vstack([text_embeddings, syn_embed])

for i in range(text_embed_with_syn.shape[0]):
    for j in range(i + 1, text_embed_with_syn.shape[0]):
        first_vector = text_embed_with_syn[i].detach().numpy().reshape(1, -1)
        second_vector = text_embed_with_syn[j].detach().numpy().reshape(1, -1)
        syn_cosine_similarities_dct[f'{i} - {j}'] = {
            "Cosine measure": cosine_similarity(first_vector, second_vector)[0][0],
            "Manhattan distance": manhattan_distances(first_vector, second_vector)[0][0],
            "Euclidian distance": euclidean_distances(first_vector, second_vector)[0][0],
        }

In [42]:
for pair, similarity in sorted(syn_cosine_similarities_dct.items(), key=lambda x: x[1]["Cosine measure"], reverse=True):
    print(f"Pair {pair} -> {similarity}")

Pair 3 - 20 -> {'Cosine measure': 0.9236032, 'Manhattan distance': 162.6966446529259, 'Euclidian distance': 7.3434963}
Pair 3 - 17 -> {'Cosine measure': 0.8810005, 'Manhattan distance': 208.67742345348233, 'Euclidian distance': 9.483934}
Pair 0 - 17 -> {'Cosine measure': 0.8714633, 'Manhattan distance': 219.12258523446508, 'Euclidian distance': 9.88745}
Pair 8 - 13 -> {'Cosine measure': 0.867718, 'Manhattan distance': 237.280664157297, 'Euclidian distance': 10.9142885}
Pair 0 - 3 -> {'Cosine measure': 0.867203, 'Manhattan distance': 210.03540116612567, 'Euclidian distance': 9.566016}
Pair 0 - 19 -> {'Cosine measure': 0.86392784, 'Manhattan distance': 212.8782062981627, 'Euclidian distance': 9.671088}
Pair 3 - 19 -> {'Cosine measure': 0.862579, 'Manhattan distance': 207.311513835215, 'Euclidian distance': 9.616859}
Pair 17 - 19 -> {'Cosine measure': 0.86007243, 'Manhattan distance': 222.3967786655412, 'Euclidian distance': 10.255882}
Pair 8 - 19 -> {'Cosine measure': 0.8469282, 'Manhatt

### Results

    RuBERT model works very slow, and it's not a good idea to use it for text vectorization